<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/w2v2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torchaudio datasets pandas

In [3]:
import torchaudio
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
from transformers import TrainingArguments, Trainer
import os

# Path to your folders
audio_folder = "FluencyBank"
label_file = "output.csv"

# Load the labels
labels_df = pd.read_csv(label_file)
labels_df['path'] = labels_df.apply(lambda row: f"FluencyBank_{int(row['EpId']):03}_{row['ClipId']}.wav", axis=1)

# Prepare dataset
def prepare_dataset(row):
    speech_array, sampling_rate = torchaudio.load(row['path'])
    return {"speech": speech_array[0], "sampling_rate": sampling_rate, "labels": row['Output']}

dataset = Dataset.from_pandas(labels_df)
dataset = dataset.map(prepare_dataset, remove_columns=["EpId", "ClipId", "Output", "path"])

# Split dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({"train": train_test_split['train'], "test": train_test_split['test']})

# Load processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base", num_labels=dataset['train'].features['labels'].num_classes)

# Preprocess data
def preprocess_data(batch):
    input_values = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], return_tensors="pt").input_values
    return {"input_values": input_values.squeeze(), "labels": batch["labels"]}

train_dataset = dataset_dict["train"].map(preprocess_data, batched=True, batch_size=8, remove_columns=dataset_dict["train"].column_names)
test_dataset = dataset_dict["test"].map(preprocess_data, batched=True, batch_size=8, remove_columns=dataset_dict["test"].column_names)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

# Train the model
trainer.train()

# You can save the model and processor with the following commands
model.save_pretrained("./wav2vec2-finetuned")
processor.save_pretrained("./wav2vec2-finetuned")

Map:   0%|          | 0/5477 [00:00<?, ? examples/s]

RuntimeError: Failed to open the input "FluencyBank_010_0.wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7fc4c40f4d87 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7fc4c40a575f in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42904 (0x7fc4aa2ca904 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x14 (0x7fc4aa2c